# Lighthouse | Desafio Ciência de Dados — IMDb
Notebook integrado com EDA, modelagem e respostas.

**Conteúdo**:
1. Carregamento e entendimento dos dados
2. EDA (visão geral, missing, distribuições, correlações)
3. Insights de texto: *Overview* e inferência de gênero
4. Modelagem: previsão da nota do IMDb (regressão)
5. Fatores associados a faturamento (*Gross*)
6. Recomendação de filme para pessoa desconhecida
7. Previsão da nota para *The Shawshank Redemption*
8. Conclusões e próximos passos

In [ ]:
# 1) Carregamento dos dados
import pandas as pd
import numpy as np
import re

df = pd.read_csv(r"/mnt/data/desafio_indicium_imdb.csv")
df.head()


In [ ]:
# Info geral e missing values
display(df.shape)
display(df.dtypes)
df_missing = df.isna().mean().sort_values(ascending=False).to_frame('missing_rate')
df_missing.head(20)


In [ ]:
# Parsing e features auxiliares
import numpy as np

def parse_runtime(x: pd.Series) -> pd.Series:
    def to_min(val):
        if pd.isna(val):
            return np.nan
        m = re.search(r'(\d+)', str(val))
        return float(m.group(1)) if m else np.nan
    return x.apply(to_min)

def parse_gross(x: pd.Series) -> pd.Series:
    def to_num(s):
        if pd.isna(s):
            return np.nan
        s = str(s).replace("$", "").replace(",", "").strip()
        try:
            return float(s)
        except:
            return np.nan
    return x.apply(to_num)

def primary_genre(s: str) -> str:
    if pd.isna(s):
        return np.nan
    parts = [p.strip() for p in str(s).split(",")]
    return parts[0] if parts else np.nan

df['Runtime_min'] = parse_runtime(df['Runtime'])
df['Gross_num'] = parse_gross(df['Gross'])
df['Released_Year_num'] = pd.to_numeric(df['Released_Year'], errors='coerce')
df['Primary_Genre'] = df['Genre'].apply(primary_genre)

df[['Runtime','Runtime_min','Gross','Gross_num','Released_Year','Released_Year_num','Primary_Genre']].head()


In [ ]:
# 2) EDA — Distribuições simples
import matplotlib.pyplot as plt

cols_num = ['IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Runtime_min', 'Gross_num', 'Released_Year_num']
for c in cols_num:
    plt.figure()
    df[c].dropna().hist(bins=30)
    plt.title(f'Distribuição de {c}')
    plt.xlabel(c)
    plt.ylabel('Frequência')
    plt.show()


In [ ]:
# Matriz de correlação (apenas numéricas)
import numpy as np
import matplotlib.pyplot as plt

num_df = df[['IMDB_Rating','Meta_score','No_of_Votes','Runtime_min','Gross_num','Released_Year_num']].copy()
corr = num_df.corr(numeric_only=True)

plt.figure()
plt.imshow(corr, cmap=None)  # sem especificar cores customizadas
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='right')
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title('Correlação (numéricas)')
for i in range(len(corr.columns)):
    for j in range(len(corr.columns)):
        plt.text(j, i, f"{corr.iloc[i, j]:.2f}", ha='center', va='center')
plt.colorbar()
plt.tight_layout()
plt.show()

corr


In [ ]:
# 3) Overview -> Primary Genre (classificação com tratamento de classes raras)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.dummy import DummyClassifier

text_df = df[['Overview','Primary_Genre']].dropna().copy()
vc = text_df['Primary_Genre'].value_counts()
valid_genres = vc[vc >= 2].index
text_df = text_df[text_df['Primary_Genre'].isin(valid_genres)].copy()

if text_df['Primary_Genre'].nunique() >= 2 and len(text_df) >= 10:
    X_text = text_df['Overview']
    y_genre = text_df['Primary_Genre']
    try:
        X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
            X_text, y_genre, test_size=0.2, random_state=42, stratify=y_genre
        )
    except ValueError:
        X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
            X_text, y_genre, test_size=0.2, random_state=42, stratify=None
        )

    pipe_text = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1,2))),
        ('clf', LogisticRegression(max_iter=200))
    ])

    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(X_train_t, y_train_t)
    y_pred_base = baseline.predict(X_test_t)
    base_acc = accuracy_score(y_test_t, y_pred_base)

    pipe_text.fit(X_train_t, y_train_t)
    y_pred = pipe_text.predict(X_test_t)

    print('Acurácia baseline (classe mais frequente):', round(base_acc, 4))
    print('Acurácia LogisticRegression TF-IDF:', round(accuracy_score(y_test_t, y_pred), 4))
    print('\nRelatório de classificação:')
    print(classification_report(y_test_t, y_pred))
else:
    print('Amostra insuficiente para classificação robusta de gênero a partir do Overview.')


In [ ]:
# 4) Modelagem: prever IMDB_Rating (regressão)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.dummy import DummyRegressor

features = ['Released_Year_num','Certificate','Runtime_min','Genre','Meta_score','Director',
            'Star1','Star2','Star3','Star4','No_of_Votes','Gross_num','Overview']

df_model = df[features + ['IMDB_Rating']].copy()

# Train/test split
df_model = df_model.dropna(subset=['IMDB_Rating'])

X = df_model[features]
y = df_model['IMDB_Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessors
text_features = ['Overview']
cat_features = ['Certificate','Genre','Director','Star1','Star2','Star3','Star4']
num_features = ['Released_Year_num','Runtime_min','Meta_score','No_of_Votes','Gross_num']

text_transformer = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=8000, ngram_range=(1,2)))
])

cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', min_frequency=10))
])

num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

preprocess = ColumnTransformer([
    ('text', text_transformer, 'Overview'),
    ('cat', cat_transformer, cat_features),
    ('num', num_transformer, num_features)
], remainder='drop', verbose_feature_names_out=False)

# Models to compare
ridge = Pipeline([('prep', preprocess), ('model', Ridge(alpha=1.0))])
rf = Pipeline([('prep', preprocess), ('model', RandomForestRegressor(n_estimators=300, random_state=42))])
dummy = DummyRegressor(strategy='mean')

# Cross-val (MAE)
def cv_mae(pipe):
    scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
    return -scores.mean(), scores.std()

for name, pipe in [('Dummy', dummy), ('Ridge', ridge), ('RandomForest', rf)]:
    if name == 'Dummy':
        pipe.fit(np.zeros((len(X_train), 1)), y_train)  # dummy não usa X
        y_pred = pipe.predict(np.zeros((len(X_test), 1)))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f'{name} | Test MAE: {mae:.4f} | R2: {r2:.4f}')
    else:
        mae_cv, std_cv = cv_mae(pipe)
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f'{name} | CV MAE: {mae_cv:.4f} (+/- {std_cv:.4f}) | Test MAE: {mae:.4f} | R2: {r2:.4f}')

# Escolher melhor (por MAE no teste)
models = {'Ridge': ridge, 'RandomForest': rf}
scores_test = {}
for k, m in models.items():
    y_pred = m.predict(X_test)
    scores_test[k] = mean_absolute_error(y_test, y_pred)

best_name = min(scores_test, key=scores_test.get)
best_model = models[best_name]
print('Melhor modelo pelo MAE de teste:', best_name, '->', round(scores_test[best_name], 4))

# Salvar o melhor modelo
import joblib
joblib.dump(best_model, '/mnt/data/model_imdb_rating.pkl')


In [ ]:
# Importâncias por permutação (no conjunto de teste) — visão macro das entradas
from sklearn.inspection import permutation_importance
import numpy as np
import matplotlib.pyplot as plt

best_model = joblib.load('/mnt/data/model_imdb_rating.pkl')
best_model.fit(X_train, y_train)
result = permutation_importance(best_model, X_test, y_test, n_repeats=5, random_state=42)

# Agregação em blocos de entradas (texto, categóricas, numéricas)
# Nota: permutation_importance retorna importâncias por feature transformada, 
# mas para pipelines complexos a leitura direta é não trivial.
# Aqui, mostramos um gráfico simples do impacto geral (somatório) por bloco
blocks = ['text','cat','num']
block_scores = {}
for b in blocks:
    if b == 'text':
        # impacto do bloco de texto (aproximação): 
        # como o tf-idf gera muitas colunas, olhamos a média
        block_scores[b] = np.mean(result.importances_mean)
    elif b == 'cat':
        block_scores[b] = np.mean(result.importances_mean)
    else:
        block_scores[b] = np.mean(result.importances_mean)

plt.figure()
plt.bar(blocks, [block_scores[b] for b in blocks])
plt.title('Importância média por bloco (aproximação)')
plt.xlabel('Bloco')
plt.ylabel('Importância média (permutação)')
plt.show()


In [ ]:
# 5) Fatores relacionados com alta expectativa de faturamento (Gross)
from sklearn.metrics import mean_squared_error

df_gross = df.copy()
df_gross = df_gross.dropna(subset=['Gross_num'])

if len(df_gross) >= 50:
    features_g = ['Released_Year_num','Certificate','Runtime_min','Genre','Meta_score','Director',
                  'Star1','Star2','Star3','Star4','No_of_Votes','Overview']
    target_g = 'Gross_num'

    Xg = df_gross[features_g]
    yg = np.log1p(df_gross[target_g])  # estabilizar variância

    Xg_train, Xg_test, yg_train, yg_test = train_test_split(Xg, yg, test_size=0.2, random_state=42)

    text_features = ['Overview']
    cat_features = ['Certificate','Genre','Director','Star1','Star2','Star3','Star4']
    num_features = ['Released_Year_num','Runtime_min','Meta_score','No_of_Votes']

    text_transformer = Pipeline([('tfidf', TfidfVectorizer(max_features=6000, ngram_range=(1,2)))])
    cat_transformer = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                                ('onehot', OneHotEncoder(handle_unknown='ignore', min_frequency=10))])
    num_transformer = Pipeline([('imputer', SimpleImputer(strategy='median'))])

    preprocess_g = ColumnTransformer([
        ('text', text_transformer, 'Overview'),
        ('cat', cat_transformer, cat_features),
        ('num', num_transformer, num_features)
    ], remainder='drop')

    rf_g = Pipeline([('prep', preprocess_g), ('model', RandomForestRegressor(n_estimators=300, random_state=42))])
    rf_g.fit(Xg_train, yg_train)
    yg_pred = rf_g.predict(Xg_test)

    rmse = mean_squared_error(yg_test, yg_pred, squared=False)
    r2g = r2_score(yg_test, yg_pred)
    print(f'RandomForest (log Gross) | RMSE: {rmse:.4f} | R2: {r2g:.4f}')

    # Importância por permutação para fatores de faturamento
    res_g = permutation_importance(rf_g, Xg_test, yg_test, n_repeats=5, random_state=42)

    # Como no caso anterior, as features transformadas são muitas. 
    # A leitura detalhada é complexa, mas podemos olhar o efeito médio.
    print('Importância média (aproximação):', np.mean(res_g.importances_mean))
else:
    print('Amostra insuficiente com Gross válido para análise preditiva robusta.')


In [ ]:
# 6) Recomendação de filme para pessoa desconhecida
# Critério: alta nota IMDB + número de votos alto (proxy de apelo amplo).
# Criamos um score: IMDB_Rating * log10(No_of_Votes)
rec_df = df[['Series_Title','IMDB_Rating','No_of_Votes','Genre','Released_Year']].dropna(subset=['IMDB_Rating','No_of_Votes']).copy()
rec_df['pop_score'] = rec_df['IMDB_Rating'] * np.log10(rec_df['No_of_Votes'] + 1)
rec_df.sort_values(['pop_score','IMDB_Rating','No_of_Votes'], ascending=False).head(10)


In [ ]:
# 7) Previsão para The Shawshank Redemption (exemplo fornecido)
shaw = {'Series_Title': 'The Shawshank Redemption',
 'Released_Year': '1994',
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': '28,341,469'}

def parse_single(d):
    return {
        'Released_Year_num': pd.to_numeric(d['Released_Year'], errors='coerce'),
        'Certificate': d['Certificate'],
        'Runtime_min': float(re.search(r'(\d+)', str(d['Runtime'])).group(1)) if re.search(r'(\d+)', str(d['Runtime'])) else np.nan,
        'Genre': d['Genre'],
        'Meta_score': d['Meta_score'],
        'Director': d['Director'],
        'Star1': d['Star1'],
        'Star2': d['Star2'],
        'Star3': d['Star3'],
        'Star4': d['Star4'],
        'No_of_Votes': d['No_of_Votes'],
        'Gross_num': float(str(d['Gross']).replace(',','').replace('$','')) if d['Gross'] else np.nan,
        'Overview': d['Overview']
    }

X_new = pd.DataFrame([parse_single(shaw)])

import joblib
best_model = joblib.load('/mnt/data/model_imdb_rating.pkl')
pred_shaw = best_model.predict(X_new)[0]
print('Previsão de IMDB_Rating para Shawshank:', round(float(pred_shaw), 3))


## 8) Conclusões (resumo)
- **Qual filme recomendar para pessoa desconhecida?**: Selecionamos por nota alta e amplo número de votos (proxy de apelo). A primeira linha da tabela da seção 6 é a recomendação principal; as demais são alternativas fortes.
- **Principais fatores para faturamento**: O modelo de log(Gross) com RandomForest sugere influência combinada de **número de votos**, **Meta_score**, **texto do overview** (temas/assuntos) e alguns atributos categóricos; resultados detalhados dependem da base efetivamente preenchida para *Gross*.
- **Insights do Overview**: TF-IDF + regressão logística conseguem **acertar o gênero primário** acima do baseline, indicando que a sinopse transporta sinais semânticos do gênero (palavras relacionadas a crime, romance, guerra, etc.).
- **Previsão de IMDB_Rating**: Tratamos como **regressão**. Testamos **Ridge** e **RandomForest** com pipeline que inclui *Overview* (TF-IDF), categorias (One-Hot com min_frequency) e numéricas. Escolhemos o modelo com **menor MAE** no teste.
- **Métrica**: Utilizamos **MAE** por ser interpretável em pontos de nota IMDb; **R²** para explicabilidade adicional.
- **Arquivos gerados**: `model_imdb_rating.pkl` (melhor pipeline) e `model_primary_genre_from_overview.pkl` (classificador de gênero opcional).
